In [ ]:
import pandas as pd
import time

# Assuming single rate for given aq, not incremental (for different usage)
# Limits on starting date based on first data point in rate table
start_time = time.time()
""""""""""""""""
CHANGE THE PATH TO WHERE YOUR EXCEL IS LOCATED
"""""""""""""""""

file = pd.ExcelFile(r'/Users/juliankautsch/PycharmProjects/GorillaAssesment/gorilla_test_data.xlsx')
RateTable = pd.read_excel(file, 'rate_table')
MeterList = pd.read_excel(file, 'meter_list')
ForecastTable = pd.read_excel(file, 'forecast_table')
MetersList = []

# this loop could be enclosed in function and run  in parallel with multiprocessing.Process
# change in how the data is stored after the loop would be required
for i in range(len(MeterList)):
    meterid = MeterList['meter_id'][i]
    aq = MeterList['aq_kwh'][i]
    ExitZone = MeterList['exit_zone'][i]
    Rates1 = RateTable[(RateTable.exit_zone == ExitZone) & (RateTable.aq_min_kwh <= aq) &
                       ((RateTable.aq_max_kwh >= aq) | (RateTable.aq_max_kwh.isnull()))]
    RateDates = Rates1['date']
    Rate = pd.DataFrame(Rates1['rate_p_per_kwh'])
    Forecast1 = ForecastTable[ForecastTable.meter_id == meterid]
    EstimatedCost = Forecast1['kwh']
    ForecastDates = Forecast1['date']
    consumption = 0
    totalCost = 0
    n = 0
    a = 0
    ConsumptionList = []
    # find starting rate - there aren't too many time periods, so quick solution
    # general case solution, could be calculated only once if all starting dates are the same
    # if this was the case also Consumption calculation could be divided into same size chunks
    while ForecastDates.iloc[0] > RateDates.iloc[n]:
        if ForecastDates.iloc[0] < RateDates.iloc[n + 1]: break
        n += 1
    starting_n = n

    # for starting conditions
    ConsumptionInPeriod = 0
    for i in RateDates:
        if n < len(RateDates) - 1:
            lastConsumption = ConsumptionInPeriod
            ConsumptionInPeriod = Forecast1.loc[(Forecast1['date'] >= RateDates.iloc[n]) &
                                                (Forecast1['date'] < RateDates.iloc[n + 1]), "kwh"].sum()
        else:
            ConsumptionInPeriod = Forecast1.loc[(Forecast1['date'] >= RateDates.iloc[n]), "kwh"].sum()
        n += 1
        ConsumptionList.append(ConsumptionInPeriod)
        if (ConsumptionInPeriod == 0) & (lastConsumption > 0) | n == len(RateDates): break

    ConsumptionDf = pd.DataFrame(ConsumptionList, columns=['kwh'])
    TotalCost = round((ConsumptionDf.values * Rate.iloc[starting_n:n].values / 100).sum(), 2)
    TotalConsumption = EstimatedCost.sum()
    MeterInfo = [meterid, TotalConsumption, TotalCost]
    MetersList.append(MeterInfo)
df = pd.DataFrame(MetersList, columns=['Meter ID', 'Total Estimated Consumption (kWh)', 'Total Cost (£)'])
print(df)
print("--- %s seconds ---" % (time.time() - start_time))